In [1]:
import pandas as pd
import os
import fnmatch
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

def get_subdirectories_with_measurements(directory: str):
    matches = []
    for root, dirs, files in os.walk(directory):
        for subdir in dirs:
            if fnmatch.fnmatch(subdir, 'measurements_*'):
                matches.append(os.path.join(root, subdir))
    return matches

def create_graphs_for_experiment(path):
    measurements = pd.read_csv(f"{path}/measurements.csv", decimal='.')

    timings = measurements.tail(2)
    timings.columns = ['timestamp']

    start = timings['timestamp'].min()
    end = timings['timestamp'].max()
    runtime = end - start

    # Omit the last two rows
    power_draws = measurements.iloc[:-2]
    power_draws.columns = ['current']

    time_between_measurements = runtime / power_draws.shape[0]

    power_draws['time'] = power_draws.index.to_series().apply(lambda index: time_between_measurements * index)
    power_draws['timestamp'] = power_draws.index.to_series().apply(lambda index: start + time_between_measurements * index)

    # Now df contains the DataFrame without the last two rows
    fmnist_timestamps = pd.read_csv(f"{path}/events.csv")
    fmnist_timestamps.columns = ['timestamp', 'label']
    timestamps_for_this_run = fmnist_timestamps[(fmnist_timestamps['timestamp'] > start) & (fmnist_timestamps['timestamp'] < end)]
    timestamps_for_this_run['time'] = timestamps_for_this_run['timestamp'].apply(lambda timestamp: timestamp - start)

    fig = px.line(power_draws, x='time', y='current')
    fig.add_hline(y=55*1000, line_dash="dot",
                annotation_text="baseline", 
                annotation_position="bottom right")

    for index, row in timestamps_for_this_run.iterrows():
        fig.add_vline(x=row['time'], line_width=3, line_color="green", annotation=dict(text=row['label'], textangle=-90))

    pio.write_image(fig, file=f"{path}/plot.png", width=1920, height=1080)

experiment_paths = get_subdirectories_with_measurements('.')
for path in experiment_paths:
    create_graphs_for_experiment(path)

/tmp/ipykernel_49574/3862834603.py:30: RuntimeWarning: divide by zero encountered in scalar divide
  time_between_measurements = runtime / power_draws.shape[0]


PermissionError: [Errno 13] Permission denied: 'measurements_fmnist_full_0527163107/plot.png'

In [ ]:
df = pd.read_csv('data/measurements_sleep_10.csv', decimal='.')

timings = df.tail(2)
timings.columns = ['timestamp']
print(timings)

start = timings['timestamp'].min()
end = timings['timestamp'].max()
runtime = end - start

# Omit the last two rows
power_draws = df.iloc[:-2]
power_draws.columns = ['current']

time_between_measurements = runtime / power_draws.shape[0]

power_draws['time'] = power_draws.index.to_series().apply(lambda index: time_between_measurements * index)

# Now df contains the DataFrame without the last two rows
print(power_draws)

FileNotFoundError: [Errno 2] No such file or directory: 'data/measurements_sleep_10.csv'